In [1]:
import mysql.connector
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from search_utils import get_embedding, update_embeddings_in_batches

c:\Users\2ndro\anaconda3\envs\nlp-thesis-similarity\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
database = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
)
cursor = database.cursor()
cursor.execute("USE nlp_thesis_similarity")

In [3]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [5]:
# First, make sure the columns exist
try:
    # Check if columns exist first
    cursor.execute("SHOW COLUMNS FROM dewey_papers LIKE 'title_embeddings_all-MiniLM-L6-v2'")
    if not cursor.fetchone():
        cursor.execute("ALTER TABLE dewey_papers ADD COLUMN `title_embeddings_all-MiniLM-L6-v2` JSON")
        
    cursor.execute("SHOW COLUMNS FROM dewey_papers LIKE 'abstract_embeddings_all-MiniLM-L6-v2'")
    if not cursor.fetchone():
        cursor.execute("ALTER TABLE dewey_papers ADD COLUMN `abstract_embeddings_all-MiniLM-L6-v2` JSON")
        
    database.commit()
    print("Columns checked and added if needed")
except Exception as e:
    print(f"Error managing columns: {e}")
    database.rollback()  # Rollback in case of error

Columns checked and added if needed


In [11]:
query = "SELECT id FROM dewey_papers WHERE abstract = ''"

cursor.execute(query)
papers = cursor.fetchall()
print(type(papers))
print(len(papers))

<class 'list'>
3441


In [4]:
# Run the update process with the loaded model
update_embeddings_in_batches(model, cursor, database)

print("Embedding process completed")

Processing 0 papers that need embeddings...
No papers need embeddings. Process complete.
Embedding process completed
